## Enviorment
The following section are used to specify the hyperparameter and the path the the software that were used. 
No need to changed if the pipeline stay the same and were applied to different data


In [141]:
##!/bin/sh
# MAKE SURE FUSION.compute_weights.R IS IN YOUR PATH
# FILL IN THESE PATHS
# For mac user, the mac version of GCTA shall be downloaded saperately, the one came with the Fusion package will not work.
GCTA="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/gcta_1.93.2beta_mac/gcta64"
PLINK="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/plink_mac_20200616/plink"
GEMMA="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/GEMMA"

# Required the customized fusion.compute_weight.mod.R script, other wise will not work
Rscp="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/FUSION.compute_weights.mod.R"
# Path to the input data
Datarep="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/Testing/Data/"
# ALTERNATIVELY: ENSURE THAT plink, gcta, gemma CAN BE CALLED FROM PATH AND REMOVE --PATH_* FLAGS BELOW

# PATH TO WORKING DIRECTORY
wd="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/Testing"

# PATH TO OUTPUT DIRECTORY (population-specific subdirs will be made)
OUT_DIR="./WEIGHTS"

# ROWS IN THE MATRIX TO ANALYZE (FOR BATCHED RUNS)
BATCH_START=1
BATCH_END=56

## Input 

This is where the actual RNA Seq data of our own should be used as input.

Note: GD462.GeneQuantRPKM.50FN.samplename.resk10.txt is the rna seq data for gene expression from the "465 lymphoblastoid cell lines from the 1000 Genomes" projects. There is 23722 genes from 462 samples. The first 4 col is "TargetID, Gene_Symbol, Chr, Coord". The rest are each samples.

GD462.GeneQuantRPKM.50FN.samplename.resk10.txt is the RNA seq data for the first 10 ppl

LDREF/1000G.EUR.*.bim contains the linkage disequilibrium info for each snp.

In [142]:
# PATH TO DIRECTORY CONTAINING LDREF DATA (FROM FUSION WEBSITE or https://data.broadinstitute.org/alkesgroup/FUSION/LDREF.tar.bz2)
LDREF="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF"
# THIS IS USED TO RESTRICT INPUT SNPS TO REFERENCE IDS ONLY

# PATH TO GEUVADIS GENE EXPRESSION MATRIX:
PRE_GEXP="GD462.hsq_succ.test.txt"
# GEUVADIS DATA WAS DOWNLOADED FROM https://www.ebi.ac.uk/arrayexpress/experiments/E-GEUV-1/files/analysis_results/

# PATH TO PREFIX FOR GEUVADIS GENOTYPES SPLIT BY CHROMOSOME
# SUBSAMPLE THESE TO THE LDREF SNPS FOR EFFICIENCY
PRE_GENO="1000G.EUR"

## Pipeline


In [148]:
# --- BEGIN SCRIPT:
cd $wd
NR="${BATCH_START}_${BATCH_END}"
mkdir -p tmp/$NR
mkdir -p hsq/$NR
mkdir -p out/$NR
# THIS IS DIRECTORY WHERE THE OUTPUT WILL GO:
mkdir $OUT_DIR

mkdir: ./WEIGHTS: File exists



Note 1: the first two column of the out.pheno shall be the same as that of the .fam file

In [171]:
# makes necessary component that are useful for follow up pipeline
cd $wd

# get the within family sample ID, need to customized for it to match those of the LDREF file
head $Datarep$PRE_GEXP | awk '{$1=$2=$3=$4=""; print substr($0,5)}' | fmt -1 > PRE_GEXPID

# make a diretory containing the per_gene file
mkdir $Datarep${PRE_GEXP%.*}_per_gene



mkdir: /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/Testing/Data/GD462.hsq_succ.test_per_gene: File exists



## SOS Pipeline prep (WIP)

In [147]:
cd $wd

# extract each gene from the data matrixs to a file
cat $Datarep$PRE_GEXP | awk -v s=$BATCH_START -v e=$BATCH_END 'NR > s && NR <= e' |  while read PARAM; do

GNAME=`echo $PARAM | awk '{ print $1 }'`
echo $PARAM > $Datarep${PRE_GEXP%.*}_per_gene/${PRE_GEXP%.*}_$GNAME.txt

done

bash: syntax error near unexpected token `done'



## Original Pipeline (Working but needs to transformed into sos pipeline)

In [145]:
cd $wd
# Loop through each gene expression phenotype in the batch (Only looking at the first 10 genes)
cat $Datarep$PRE_GEXP | awk -v s=$BATCH_START -v e=$BATCH_END 'NR > s && NR <= e' |  while read PARAM; do
# Get the gene positions +/- 500kb
CHR=`echo $PARAM | awk '{ print $3 }'`
P0=`echo $PARAM | awk '{ print $4 - 0.5e6 }'`
P1=`echo $PARAM | awk '{ print $4 + 0.5e6 }'`
GNAME=`echo $PARAM | awk '{ print $1 }'`

OUT="tmp/$NR/$PRE_GEXP.$GNAME"
echo $GNAME $CHR $P0 $P1

## Pull out the current gene expression phenotype (Note 1 )
echo $PARAM | tr ' ' '\n' | tail -n+5 | paste PRE_GEXPID PRE_GEXPID - > $OUT.pheno
## Get the locus genotypes for all samples and set current gene expression as the phenotype
$PLINK --bfile $LDREF/$PRE_GENO.$CHR --pheno $tmpdir$OUT.pheno --make-bed --out $tmpdir$OUT --chr $CHR --from-bp $P0 --to-bp $P1 --extract $LDREF/1000G.EUR.$CHR.bim --keep $tmpdir$OUT.pheno --allow-no-sex
## Process all samples together (for reference purposes only since this is mult-ethnic data)
mkdir $OUT_DIR/ALL
FINAL_OUT="$OUT_DIR/ALL/ALL.$GNAME"
#echo $FINAL_OUT
#
#echo $OUT.tmp

Rscript $Rscp --bfile $OUT --tmp $OUT.tmp --out $FINAL_OUT --verbose 0 --save_hsq --PATH_gcta $GCTA --PATH_gemma $GEMMA --models blup,lasso,top1,enet

#
## Append heritability output to hsq file
cat $FINAL_OUT.hsq >> hsq/$NR.hsq
#
## Clean-up just in case
rm -f $FINAL_OUT.hsq $OUT.tmp.*
#
# Remove all intermediate files
echo "end of circle"

# GO TO THE NEXT GENE
done




ENSG00000136237.12 7 21896763 22896763
mkdir: ./WEIGHTS/ALL: File exists
ERROR: tmp/1_56/GD462.hsq_succ.test.txt.ENSG00000136237.12.bed input file does not exist
end of circle
ENSG00000225267.1 21 31136199 32136199
mkdir: ./WEIGHTS/ALL: File exists
ERROR: tmp/1_56/GD462.hsq_succ.test.txt.ENSG00000225267.1.bed input file does not exist
end of circle
ENSG00000259382.1 6 31653659 32653659
mkdir: ./WEIGHTS/ALL: File exists
ERROR: tmp/1_56/GD462.hsq_succ.test.txt.ENSG00000259382.1.bed input file does not exist
end of circle
ENSG00000124374.8 2 70954213 71954213
mkdir: ./WEIGHTS/ALL: File exists
ERROR: tmp/1_56/GD462.hsq_succ.test.txt.ENSG00000124374.8.bed input file does not exist
cat: ./WEIGHTS/ALL/ALL.ENSG00000124374.8.hsq: No such file or directory
end of circle
ENSG00000250443.2 3 196733857 197733857
mkdir: ./WEIGHTS/ALL: File exists
ERROR: tmp/1_56/GD462.hsq_succ.test.txt.ENSG00000250443.2.bed input file does not exist
cat: ./WEIGHTS/ALL/ALL.ENSG00000250443.2.hsq: No such file or direc

In [15]:
wget https://github.com/cumc/neuro-twas/blob/master/WIP/pipeline_prototype/Fusion.compute_weights_testing.ipynb

--2020-07-13 14:31:25--  https://github.com/cumc/neuro-twas/blob/master/WIP/pipeline_prototype/Fusion.compute_weights_testing.ipynb
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-07-13 14:31:26 ERROR 404: Not Found.




In [120]:
cd tmp

In [73]:
cd tmp/1_2/
ls
cat GD462.hsq_succ.test.txt.ENSG00000136237.12..tmp.cv.log

'utf-8' codec can't decode byte 0xa4 in position 1: invalid start byte


In [74]:
rm -r *

'utf-8' codec can't decode byte 0x8c in position 3: invalid start byte


In [72]:
cat 1_10.hsq | grep . -c

cat: 1_10.hsq: No such file or directory
0



In [26]:
echo $pop

In [123]:
head /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF/1000G.10.bed

head: /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF/1000G.10.bed: No such file or directory



/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF.ID



patential working genes
ENSG00000225267.1
ENSG00000259382.1
ENSG00000135046.9

In [25]:
$OUT

# Association Test
The following section is for the assoc test, which is ready to use off the shelf, tne triky part is getting the weights

In [91]:
Rscr2="<>"
# This is the new input file
sumstats="<>"
weights=FINAL_OUT
weights_dir=$wd/WEIGHTS/
ref_ld_chr=$LDREF/PRE_GENO
chr="<>"


In [ ]:
Rscript $Rscr2 \
--sumstats $sumstats \
--weights $weights \
--weights_dir $weights_dir \
--ref_ld_chr $LDREF/PRE_GENO \
--chr 22 \
--out PGC2.SCZ.22.dat